In [1]:
import torch.nn as nn
import torch
import pandas as pd
import numpy as np
import json
import random
from openai import OpenAI
from preprocess import option_prompt_generate,preprocess_data,behavioral_embedding 
#preprocess中暂时废弃client，text embedding，behavioral_embedding_model
from Data_loader import create_data_loaders
from model import TextDecisionModel
import matplotlib.pyplot as plt

In [2]:
c13k_problems_raw = pd.read_json("data/c13k_problems.json", orient='index')
# c13k_problems_raw = pd.read_json("./result/c13k_problems.json", orient='index')
c13k_problems = preprocess_data(c13k_problems_raw)


In [3]:
client = OpenAI(
    base_url='https://api.tutujin.com/v1',
    api_key='sk-O1OLuehmNmUpIbgN4e8dCdCb4299450e90F02a3845E6E8F0') #add you own API key

问题embedding

In [4]:
# 创建问题文本
prompt_list = []  
for i in range(c13k_problems.shape[0]): #逐条处理
    tmp_data = c13k_problems.iloc[i,:]
    tmp_prompt = option_prompt_generate(tmp_data['p'],tmp_data['v'])
    tmp_prompt = np.array(tmp_prompt, dtype ='object')
    prompt_list.append(tmp_prompt)

# 循环结束后，一次性将所有prompt合并为numpy数组
prompt_dataset = np.vstack(prompt_list)

# 如果需要保持二维结构且列数为1，可以添加以下代码：
if len(prompt_dataset.shape) == 1:
    prompt_dataset = prompt_dataset.reshape(-1, 1)

# 优化后时间缩短：7.9s to 1.0s

In [27]:
# 批量获取文本嵌入
input_texts = prompt_dataset[:, 0].tolist()  # 转换为Python列表
batch_size = 1000  # 根据API限制和系统资源调整合适的批量大小
embeddings_batches = []

for i in range(0, len(input_texts), batch_size):
    batch_texts = input_texts[i:i + batch_size]
    
    tmp_embeddings_batch = client.embeddings.create(
        model='text-embedding-ada-002',
        input=batch_texts,
        # dimensions=1536
        )
        
    # 将批量结果添加到列表中
    embeddings_batches.append(tmp_embeddings_batch.data)
    print("now is batch",i+batch_size)

# 合并所有批次的嵌入结果
all_embeddings = [emb.embedding for batch in embeddings_batches for emb in batch]

# 将嵌入结果转换为numpy数组
text_problem_embeddings = np.array(all_embeddings, dtype=np.float32).reshape(prompt_dataset.shape[0], 1536)

# batch_size = 5时，时间缩短35.8s to 5.0s

now is batch 1000
now is batch 2000
now is batch 3000
now is batch 4000
now is batch 5000
now is batch 6000
now is batch 7000
now is batch 8000
now is batch 9000
now is batch 10000
now is batch 11000
now is batch 12000
now is batch 13000
now is batch 14000
now is batch 15000
now is batch 16000
now is batch 17000
now is batch 18000
now is batch 19000
now is batch 20000
now is batch 21000
now is batch 22000
now is batch 23000
now is batch 24000
now is batch 25000
now is batch 26000
now is batch 27000
now is batch 28000
now is batch 29000
now is batch 30000


In [28]:
np.save("./result/refresh_text_problem_embeddings.npy", text_problem_embeddings)
np.load('./result/refresh_text_problem_embeddings.npy')

array([[ 0.01751892, -0.00901634, -0.00247082, ..., -0.01165573,
         0.00282884, -0.01442357],
       [-0.01565006, -0.02233857, -0.00347815, ..., -0.00338671,
         0.01908576, -0.01930784],
       [ 0.01596104, -0.03506856, -0.01965883, ..., -0.01489876,
        -0.00076729, -0.02201197],
       ...,
       [ 0.00505597, -0.02872606, -0.00178881, ..., -0.00494427,
         0.0136797 , -0.02304918],
       [ 0.01087554, -0.02887958, -0.00481435, ..., -0.00595021,
        -0.0040669 , -0.01366949],
       [ 0.01734488, -0.02502469, -0.0001904 , ..., -0.0190428 ,
         0.00752308, -0.02537734]], dtype=float32)

In [5]:
#origin = np.load('./result/text_problem_embeddings.npy')
text_problem_embeddings = np.load('./result/refresh_text_problem_embeddings.npy')

决策embedding

In [6]:
# 初始行为嵌入向量列表
behavioral_embedding_dataset = []

for i in range(c13k_problems.shape[0]):
    # 提取单个行为数据
    tmp_data = c13k_problems.iloc[i, :]
    # 计算行为嵌入向量
    tmp_behavioral_embedding = behavioral_embedding(tmp_data['p'], tmp_data['v'])
    # 将行为嵌入向量添加到列表中
    behavioral_embedding_dataset.append(tmp_behavioral_embedding)

# 将嵌入向量列表转换为numpy数组，并指定数据类型为float32
behavioral_embedding_dataset = np.array(behavioral_embedding_dataset, dtype='float32')

In [7]:
np.save("result/refresh_behavioral_embeddings.npy", behavioral_embedding_dataset)

In [8]:
#origin = np.load('./result/c13k_behav_embeddings.npy')
behavioral_embeddings = np.load('./result/refresh_behavioral_embeddings.npy')

训练模型

In [13]:
batch_size= 256
train_dataloader, val_dataloader,test_dataloader = create_data_loaders(text_problem_embeddings, behavioral_embeddings, batch_size=batch_size, test_size = 0.2, contrastive = False, false_scale = 2, scale = 'outcome_scaling')

In [15]:
class CosineContrastiveLoss(nn.Module):
    def __init__(self, margin=0.9):
        super(CosineContrastiveLoss, self).__init__()
        self.margin = margin

    def forward(self, output1, output2, label):
        # Cosine similarity
        cosine_sim = nn.functional.cosine_similarity(output1, output2)
        
        # For similar pairs: we want the negative of (1 - cosine_sim) to make it closer to 1
        # For dissimilar pairs: we take max(0, cosine_sim - margin) to push them apart until a certain margin
        loss_contrastive = torch.mean(torch.mean((label) * torch.pow(1 - cosine_sim, 2)) +
                                      torch.mean((1-label) * torch.pow(torch.clamp(cosine_sim - self.margin, min=0.0), 2)))
        return loss_contrastive
    
class FeatureLoss(nn.Module):

    def __init__(self):
        super(FeatureLoss, self).__init__()
        
    
    def forward(self, output1, output2):
        # Standardize each dimension
        # std1 = torch.std(output1, dim=0, keepdim=True) + 1e-8  # Avoid division by zero
        # mean1 = torch.mean(output1, dim=0, keepdim=True)
        # output1 = (output1 - mean1) / std1
        
        # std2 = torch.std(output2, dim=0, keepdim=True) + 1e-8  # Avoid division by zero
        # mean2 = torch.mean(output2, dim=0, keepdim=True)
        # output2 = (output2 - mean2) / std2
        
        feature_loss = torch.square(output1 - output2)
        return feature_loss

In [16]:
# Define the model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model = JointEmbedding().to(device)
model = TextDecisionModel().to(device)
# Define the loss, optimizer, etc.
criterion = FeatureLoss()
# criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

num_epochs = 200
cosine_similarity_threshold = 0.9 # This can be adjusted based on your needs

feature_losses = {f"dim_{i}": [] for i in range(12)}
val_feature_losses = {f"dim_{i}": [] for i in range(12)}

In [17]:
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    # true_pairs_similarity = []
    # false_pairs_similarity = []
    # For storing dimensional losses in an epoch
    epoch_dimensional_losses = {f"dim_{i}": [] for i in range(12)}
    val_epoch_dimensional_losses = {f"dim_{i}": [] for i in range(12)}
    
    for batch in train_dataloader:
        text_embeds = batch["text_embedding"].to(device)
        decision_embeds = batch["decision_embedding"].to(device)
        labels = batch["label"].to(device)

        optimizer.zero_grad()
        # text_outputs, behavioral_outputs = model(text_embeds, decision_embeds)
        
        decision_pred = model(text_embeds)
        #dimensional-wise loss to track the training
        feature_loss = criterion(decision_pred, decision_embeds)
        #MSE loss indeed
        loss = torch.mean(torch.sum(feature_loss, dim=1))
        # loss = criterion(text_outputs, behavioral_outputs,labels)
        loss.backward()
        optimizer.step()
        
        for i in range(12):
            epoch_dimensional_losses[f"dim_{i}"].append(feature_loss[:, i].mean().item())
        
        total_loss += loss.item()
    
    # Store the average dimensional loss for this epoch
    for i in range(12):
        feature_losses[f"dim_{i}"].append(np.mean(epoch_dimensional_losses[f"dim_{i}"]))
        
    print(f"Epoch {epoch + 1}/{num_epochs} - Loss: {total_loss/len(train_dataloader)}")

    model.eval()

    with torch.no_grad():
        correct_pairs = 0
        total_pairs = 0
        total_val_loss = 0
        # true_pairs_val_similarity = []
        # false_pairs_val_similarity = []

        for batch in test_dataloader:
            text_emb = batch["text_embedding"].to(device)
            decision_emb = batch["decision_embedding"].to(device)
            label = batch["label"].to(device)
    
            text_proj = model(text_emb)
            cosine_sim = nn.functional.cosine_similarity(text_proj, decision_emb)
            # loss = criterion(text_proj, decision_proj, label)
            decision_pred = model(text_embeds)
            val_feature_loss = criterion(decision_pred, decision_embeds)
            loss = torch.mean(torch.sum(val_feature_loss, dim=1))
            total_val_loss += loss.item()

            # # Calculate similarity for validation
            # true_pairs_val_similarity.extend(cosine_sim[label == 1].cpu().numpy())
            # false_pairs_val_similarity.extend(cosine_sim[label == 0].cpu().numpy())

            # # Check pairs based on cosine similarity
            correct_pairs += ((cosine_sim > cosine_similarity_threshold) & (label == 1)).sum().item()
            correct_pairs += ((cosine_sim <= cosine_similarity_threshold) & (label == 0)).sum().item()
            total_pairs += label.size(0)
            
            for i in range(12):
                val_epoch_dimensional_losses[f"dim_{i}"].append(val_feature_loss[:, i].mean().item())
        
        # Store the average dimensional loss for this epoch
        for i in range(12):
            val_feature_losses[f"dim_{i}"].append(np.mean(val_epoch_dimensional_losses[f"dim_{i}"]))
                
        # true_avg_val_sim = np.mean(true_pairs_val_similarity)
        # false_avg_val_sim = np.mean(false_pairs_val_similarity)

        print(f"Epoch {epoch + 1}/{num_epochs} - Validation Loss: {total_val_loss/len(test_dataloader)}")
        # print(f"Average True Matching Similarity (Validation): {true_avg_val_sim:.2f}")
        # print(f"Average False Matching Similarity (Validation): {false_avg_val_sim:.2f}")
        # print(f"Difference (Validation): {true_avg_val_sim - false_avg_val_sim:.2f}\n")
        print(f"Validation Accuracy: {correct_pairs / total_pairs * 100:.2f}%")

KeyError: 'label'

In [ ]:
model_name = model.__class__.__name__ + '.pth'
torch.save(model.state_dict(), 'result/' + model_name)

In [ ]:
# Visualizing the losses
plt.figure(figsize=(6,4))
for i in range(12):
    plt.plot(feature_losses[f"dim_{i}"], label=f"Dimension {i}")
plt.legend()
plt.title("Dimension-wise Training Losses over Epochs")
plt.xlabel("Epochs")
plt.ylabel("Loss")
# Save the plot with a specific DPI
plt.savefig('pic/training_loss.png', dpi=300)  # 300 DPI is a common high-resolution setting
plt.show()

In [ ]:
# Visualizing the losses
plt.figure(figsize=(6,4))
for i in range(12):
    plt.plot(val_feature_losses[f"dim_{i}"], label=f"Dimension {i}")
plt.legend()
plt.title("Dimension-wise Validation Losses over Epochs")
plt.xlabel("Epochs")
plt.ylabel("Loss")
# Save the plot with a specific DPI
plt.savefig('pic/val_loss.png', dpi=300)  # 300 DPI is a common high-resolution setting
plt.show()

In [ ]:
#test by hand 
test_v = [1000, 0 ]
test_p = [0.5,0.5]
test_behavioral_embedding = behavioral_embedding(test_p, test_v)
test_behavioral_embedding[[0,1,2,3,4,10]] =  test_behavioral_embedding[[0,1,2,3,4,10]]/1000
test_text = option_prompt_generate(test_p, test_v)
print(f"test text : {test_text}")
tmp_embeddings = client.embeddings.create(
            model='text-embedding-3-large',
            input=test_text,
            dimensions=1536)
tmp_text_embeddings = tmp_embeddings.data[0].embedding
tmp_text_embeddings = torch.tensor(tmp_text_embeddings).to(device)
model.eval()
with torch.no_grad():
    tmp_text_shared_embedding = model(tmp_text_embeddings )

In [ ]:
print(f"test behavioral embedding: {test_behavioral_embedding}")
print(f"test text shared embedding: {tmp_text_shared_embedding}")